In [1]:
# 📌 Importing Required Libraries


import requests, pandas as pd

In [2]:
# 📌 Importing Required Libraries


from bs4 import BeautifulSoup

In [3]:
# 🌐 Sending an HTTP GET Request to Fetch Data
response = requests.get('https://www.mcdonalds.com/us/en-us/full-menu.html')

In [4]:
# ✅ Checking the Response Status Code

response.status_code

200

In [5]:
# 📌 Processing Data Step

response.content

b'\n\n<!DOCTYPE HTML>\n<html lang="en-US">\n    <head>\n    \t\n\n    \t\n    <meta charset="UTF-8"/>\n    <title>McDonald\xe2\x80\x99s Menu: Our Full McDonald\xe2\x80\x99s Food Menu | McDonald\xe2\x80\x99s</title>\n    \n    <meta name="description" content="Before your next McDonald\xe2\x80\x99s trip, check out our full McDonald\'s Menu. Featuring breakfast menu favorites, burgers, & more! Order today using McDelivery\xc2\xae!"/>\n    <meta name="template" content="category"/>\n    <meta name="viewport" content="width=device-width, initial-scale=1"/>\n    \n\n\n<meta name="keywords"/>\n<meta name="title" content="McDonald\xe2\x80\x99s Menu: Our Full McDonald\xe2\x80\x99s Food Menu | McDonald\xe2\x80\x99s"/>\n\n\n\n<meta property="og:type" content="website"/>\n<meta property="og:url" content="https://www.mcdonalds.com/us/en-us/full-menu.html"/>\n\n<meta property="og:title" content="McDonald\xe2\x80\x99s Menu: Our Full McDonald\xe2\x80\x99s Food Menu | McDonald\xe2\x80\x99s"/>\n<meta n

In [6]:
# 📌 Processing Data Step

soup = BeautifulSoup(response.content, 'html5lib')

In [7]:
# 📌 Processing Data Step

soup.prettify()

'<!DOCTYPE html>\n<html lang="en-US">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   McDonald’s Menu: Our Full McDonald’s Food Menu | McDonald’s\n  </title>\n  <meta content="Before your next McDonald’s trip, check out our full McDonald\'s Menu. Featuring breakfast menu favorites, burgers, &amp; more! Order today using McDelivery®!" name="description"/>\n  <meta content="category" name="template"/>\n  <meta content="width=device-width, initial-scale=1" name="viewport"/>\n  <meta name="keywords"/>\n  <meta content="McDonald’s Menu: Our Full McDonald’s Food Menu | McDonald’s" name="title"/>\n  <meta content="website" property="og:type"/>\n  <meta content="https://www.mcdonalds.com/us/en-us/full-menu.html" property="og:url"/>\n  <meta content="McDonald’s Menu: Our Full McDonald’s Food Menu | McDonald’s" property="og:title"/>\n  <meta content="category" name="pageType"/>\n  <meta content="Before your next McDonald’s trip, check out our full McDonald\'s Menu. Featuring breakfast menu fa

In [8]:
# 📌 Processing Data Step

menu = soup.find('div',class_="menu-categories aem-GridColumn aem-GridColumn--default--12")

In [9]:
# 📌 Processing Data Step

menu = menu.find('div')

In [10]:
# 📌 Processing Data Step
menu

<div>


	

		<div class="cmp-title">
            <h2 class="cmp-title__text">Breakfast</h2>
        </div>

		<ul class="cmp-category__row">
            
				



	<li class="cmp-category__item" data-product-id="200300">
		<a aria-labelledby="menu-categories-c27acea84f-1b49d5c0bc_flag menu-categories-c27acea84f-1b49d5c0bc_name menu-categories-c27acea84f-1b49d5c0bc_image" class="cmp-category__item-link" data-cmp-clickable="" data-cmp-data-layer='{"menu-categories-c27acea84f-1b49d5c0bc":{"linkPosition":"full-menu:Bacon, Egg  &amp; Cheese Biscuit:200300:1","xdm:linkURL":"/us/en-us/product/bacon-egg-cheese-biscuit.html","dc:title":"Bacon, Egg  &amp; Cheese Biscuit"}}' href="/us/en-us/product/bacon-egg-cheese-biscuit.html">
			<div aria-hidden="true" class="cmp-category__details">
				<figure>
						
                        
                        <!-- Mobile -->
                        <source media="(max-width: 767px)" srcset="https://s7d1.scene7.com/is/image/mcdonalds/DC_202405_0085_Baco

In [11]:
# 📌 Making list of the id of the products
list_id = []
for product_id in menu.find_all('li', class_= 'cmp-category__item'):
    list_id.append(product_id['data-product-id'])


In [12]:
# 📌 Function to return the index of element in dict

def find_element(string, df):
    for idx,element in enumerate(df):
        if element['name'] == string:
            return idx
    else:
        return 'None'

In [13]:
# 🌐 Sending an HTTP GET Request to Fetch Data

table = {   'Category':[],
            'Item':[],
            'Calories':[],
            'Protein':[],
            'Carbohydrates':[],
            'Total Fat':[],
            'Sodium':[],
            'Saturated Fat':[],
            'Cholesterol':[],
            'Dietary Fiber':[],
            'Total Sugars':[],
        }


# bulid up a list of 
for id in list_id:
   link1 = f'https://www.mcdonalds.com/dnaapp/itemDetails?country=US&language=en&showLiveData=true&item={id}&specialCall=true'
   link2 = f'https://www.mcdonalds.com/dnaapp/itemCollectionDetails?country=US&language=en&showLiveData=true&item={id}&specialCall=true'
   api_response = requests.get(link1)    
   if api_response.status_code == 200:
      data = api_response.json()['item']
      table['Item'].append(data['item_marketing_name'])
      try:
         table['Category'].append(data['categories']['category'][0]['name'])
      except:
         table['Category'].append(table['Category'][-1]) # To handle some rare cases' categories
      try:
         temp = data['nutrient_facts']['nutrient'] # To handle some cases
      except:
         api_response = requests.get(link2) # Some items consist of some other element so it need another api link 
         data = api_response.json()['meal_item']
         temp = data['collective_nutrition']['nutrient_facts']['nutrient'] 
      keys_list =(list(table.keys())[2:]) # starting from 3 element(2nd index) cause we've added first 2 (category and item name)



      # Adding every Nutrition Element value and unit of measure(uom)
      for key in keys_list:
         idx = find_element(key, temp)
         if idx == 'None' : table[key].append(0)
         else:
            table[key].append(temp[idx]['value'] + temp[idx]['uom'])
   else:
      print('Status Code NOT 200')

In [17]:
# 📌 Data frame of table
df = pd.DataFrame(table)
df

,Category,Item,Calories,Protein,Carbohydrates,Total Fat,Sodium,Saturated Fat,Cholesterol,Dietary Fiber,Total Sugars
0,Breakfast,"Bacon, Egg & Cheese Biscuit",460 Cal.,17g,39g,26g,1330mg,13g,215mg,2g,3g
1,Breakfast,Egg McMuffin®,310 Cal.,17g,30g,13g,770mg,6g,250mg,2g,3g
2,Breakfast,Sausage McMuffin®,400 Cal.,14g,29g,26g,760mg,10g,55mg,2g,2g
3,Breakfast,Sausage McMuffin® with Egg,480 Cal.,20g,30g,31g,830mg,12g,280mg,2g,2g
4,Breakfast,Sausage Biscuit,460 Cal.,11g,37g,30g,1090mg,13g,35mg,2g,2g
...,...,...,...,...,...,...,...,...,...,...,...
101,Beverages,Honest Kids® Appley Ever After® Organic Juice ...,35 Cal.,0g,8g,0g,15mg,0g,0mg,0g,8g
102,Beverages,1% Low Fat Milk Jug,100 Cal.,8g,12g,2g,80mg,1.5g,10mg,0g,11g
103,Beverages,Reduced Sugar* Low Fat Chocolate Milk Jug,130 Cal.,9g,18g,2.5g,85mg,1.5g,10mg,0g,14g
104,Beverages,Hot Chocolate (small),360 Cal.,11g,51g,12g,170mg,8g,45mg,1g,45g


In [18]:
df.to_excel('mcscrap.xlsx', index=False)